In [1]:
from kamgon import *

In [2]:
class UniversitySchedulingProblem(CSPProblem):
    """
    ระบบจัดตารางเรียนมหาวิทยาลัยที่ครอบคลุมข้อจำกัดที่ซับซ้อน
    รวมถึงการจัดสรรอาจารย์ ห้องเรียน และการหลีกเลี่ยงการทับซ้อนของรายวิชา
    """
    
    def __init__(self, courses: List[str], instructors: Dict[str, List[str]], 
                 rooms: List[str], time_slots: List[str], 
                 room_capacity: Dict[str, int], course_enrollment: Dict[str, int],
                 instructor_availability: Dict[str, List[str]],
                 room_requirements: Dict[str, List[str]]):
        """
        เริ่มต้นระบบจัดตารางเรียนมหาวิทยาลัย
        
        Args:
            courses: รายการรายวิชา
            instructors: รายการอาจารย์ที่สามารถสอนแต่ละวิชาได้
            rooms: รายการห้องเรียน
            time_slots: รายการช่วงเวลา
            room_capacity: ความจุของแต่ละห้อง
            course_enrollment: จำนวนนักเรียนที่ลงทะเบียนแต่ละวิชา
            instructor_availability: ช่วงเวลาที่อาจารย์สามารถสอนได้
            room_requirements: ความต้องการพิเศษของห้องสำหรับแต่ละวิชา
        """
        self.instructors = instructors
        self.rooms = rooms
        self.time_slots = time_slots
        self.room_capacity = room_capacity
        self.course_enrollment = course_enrollment
        self.instructor_availability = instructor_availability
        self.room_requirements = room_requirements
        
        # สร้าง domain ที่เป็น combination ของอาจารย์ เวลา และห้อง
        domains = {}
        for course in courses:
            possible_assignments = []
            available_instructors = instructors.get(course, [])
            required_rooms = room_requirements.get(course, rooms)
            
            for instructor in available_instructors:
                available_times = instructor_availability.get(instructor, time_slots)
                for time in available_times:
                    for room in required_rooms:
                        if (room in self.room_capacity and 
                            self.room_capacity[room] >= course_enrollment.get(course, 0)):
                            possible_assignments.append((instructor, time, room))
            
            domains[course] = possible_assignments
        
        super().__init__(courses, domains)
    
    def is_consistent(self, variable: str, value: Any, assignment: Dict[str, Any]) -> bool:
        """
        ตรวจสอบข้อจำกัดของการจัดตารางเรียน
        
        Args:
            variable: ชื่อรายวิชา
            value: tuple ของ (อาจารย์, เวลา, ห้อง)
            assignment: การจัดตารางที่มีอยู่แล้ว
            
        Returns:
            True ถ้าการจัดตารางนี้ไม่ขัดแย้งกับข้อจำกัด
        """
        instructor, time, room = value
        
        for assigned_course, assigned_value in assignment.items():
            if assigned_course == variable:
                continue
                
            assigned_instructor, assigned_time, assigned_room = assigned_value
            
            # ข้อจำกัดที่ 1: อาจารย์คนเดียวไม่สามารถสอนหลายวิชาในเวลาเดียวกันได้
            if instructor == assigned_instructor and time == assigned_time:
                return False
            
            # ข้อจำกัดที่ 2: ห้องเดียวไม่สามารถใช้หลายวิชาในเวลาเดียวกันได้
            if room == assigned_room and time == assigned_time:
                return False
        
        return True
    
    def get_schedule_summary(self, assignment: Dict[str, Any]) -> Dict[str, List[str]]:
        """
        สร้างสรุปตารางเรียนตามช่วงเวลา
        
        Args:
            assignment: การจัดตารางที่สมบูรณ์
            
        Returns:
            Dictionary ที่จัดกลุ่มตารางตามช่วงเวลา
        """
        schedule = {time: [] for time in self.time_slots}
        
        for course, (instructor, time, room) in assignment.items():
            schedule[time].append(f"{course} ({instructor}) - {room}")
        
        return schedule

In [3]:
# การใช้งาน University Scheduling Problem
if __name__ == "__main__":
    print("=== ระบบจัดตารางเรียนมหาวิทยาลัย ===")
    
    # ข้อมูลตัวอย่าง
    courses = ['คณิตศาสตร์ 1', 'ฟิสิกส์ 1', 'เคมี 1', 'โปรแกรมมิ่ง 1', 'สถิติ 1']
    
    instructors = {
        'คณิตศาสตร์ 1': ['อ.สมชาย', 'อ.สุชาติ'],
        'ฟิสิกส์ 1': ['อ.วิรัช', 'อ.นันทิดา'],
        'เคมี 1': ['อ.ปิยะ', 'อ.สุชาติ'],
        'โปรแกรมมิ่ง 1': ['อ.ทรงพล', 'อ.อรุณ'],
        'สถิติ 1': ['อ.สมชาย', 'อ.วิรัช']
    }
    
    rooms = ['ห้อง101', 'ห้อง102', 'ห้อง201', 'ห้องแล็บ1', 'ห้องแล็บ2']
    time_slots = ['08:00-10:00', '10:00-12:00', '13:00-15:00', '15:00-17:00']
    
    room_capacity = {
        'ห้อง101': 50, 'ห้อง102': 40, 'ห้อง201': 60,
        'ห้องแล็บ1': 30, 'ห้องแล็บ2': 30
    }
    
    course_enrollment = {
        'คณิตศาสตร์ 1': 45, 'ฟิสิกส์ 1': 35, 'เคมี 1': 30,
        'โปรแกรมมิ่ง 1': 25, 'สถิติ 1': 40
    }
    
    instructor_availability = {
        'อ.สมชาย': ['08:00-10:00', '10:00-12:00', '13:00-15:00'],
        'อ.สุชาติ': ['10:00-12:00', '13:00-15:00', '15:00-17:00'],
        'อ.วิรัช': ['08:00-10:00', '13:00-15:00', '15:00-17:00'],
        'อ.นันทิดา': ['08:00-10:00', '10:00-12:00', '15:00-17:00'],
        'อ.ปิยะ': ['08:00-10:00', '10:00-12:00', '13:00-15:00'],
        'อ.ทรงพล': ['10:00-12:00', '13:00-15:00', '15:00-17:00'],
        'อ.อรุณ': ['08:00-10:00', '10:00-12:00', '13:00-15:00']
    }
    
    room_requirements = {
        'คณิตศาสตร์ 1': ['ห้อง101', 'ห้อง102', 'ห้อง201'],
        'ฟิสิกส์ 1': ['ห้อง101', 'ห้อง102', 'ห้อง201'],
        'เคมี 1': ['ห้องแล็บ1', 'ห้องแล็บ2'],
        'โปรแกรมมิ่ง 1': ['ห้องแล็บ1', 'ห้องแล็บ2'],
        'สถิติ 1': ['ห้อง101', 'ห้อง102', 'ห้อง201']
    }
    
    university_scheduling = UniversitySchedulingProblem(
        courses, instructors, rooms, time_slots, room_capacity,
        course_enrollment, instructor_availability, room_requirements
    )
    
    # แก้ปัญหาการจัดตารางเรียน
    solution = backtracking_search(university_scheduling, verbose=False)
    
    if solution:
        print("การจัดตารางเรียนสำเร็จ:")
        schedule_summary = university_scheduling.get_schedule_summary(solution)
        
        for time_slot, courses_in_slot in schedule_summary.items():
            print(f"\n{time_slot}:")
            if courses_in_slot:
                for course_info in courses_in_slot:
                    print(f"  {course_info}")
            else:
                print("  ไม่มีรายวิชา")
    else:
        print("ไม่สามารถจัดตารางเรียนได้ด้วยข้อจำกัดที่กำหนด")

=== ระบบจัดตารางเรียนมหาวิทยาลัย ===
การจัดตารางเรียนสำเร็จ:

08:00-10:00:
  คณิตศาสตร์ 1 (อ.สมชาย) - ห้อง101
  เคมี 1 (อ.ปิยะ) - ห้องแล็บ1
  ฟิสิกส์ 1 (อ.วิรัช) - ห้อง102

10:00-12:00:
  โปรแกรมมิ่ง 1 (อ.ทรงพล) - ห้องแล็บ1
  สถิติ 1 (อ.สมชาย) - ห้อง101

13:00-15:00:
  ไม่มีรายวิชา

15:00-17:00:
  ไม่มีรายวิชา
